In [1]:
import tensorflow as tf
import numpy as np

In [2]:
EPOCHS = 10

#residual unit 구현

In [3]:
class ResidualUnit(tf.keras.Model):

    def __init__(self, filter_in, filter_out, kernel_size):
        super(ResidualUnit,self).__init__()
        #gradient highway 부분
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filter_out,kernel_size,padding = 'same')

        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filter_out,kernel_size,padding = 'same')

        #여기서 더해주려면 filter_in과 filter_out이 같아야 더해줄수 있다.
        #identity 설정
        if filter_in == filter_out:
            self.identity = lambda x: x
        else:
            self.identity = tf.keras.layers.Conv2D(filter_out,(1,1),padding = 'same')


    def call(self,x,training = False, mask =None):
        h = self.bn1(x, training = training)
        h = tf.nn.relu(h)
        h = self.conv1(h)

        h = self.bn2(h, training = training)
        h = tf.nn.relu(h)
        h = self.conv2(h)

        #여기서 identity를 사용
        return self.identity(x) + h

#residual layer 구현

In [4]:
class ResnetLayer(tf.keras.Model):
    def __init__(self,filter_in,filters, kernel_size):
        super(ResnetLayer,self).__init__()
        #16,[32,32,32,32] 이렇게 residual unit 
        self.sequence = list()
        #filter_in과 filters를 통해 쌍을 만들어준다.
        for f_in, f_out in zip([filter_in]+list(filters),filters):
            #16,[32,32,32]
            #zip ([16,32,32,32],[32,32,32]) 더 작은걸 기준으로 zip하니까 
            #이렇게 하면 쭉이어져서 바로뒤에꺼랑 바로바로 연결됨
            self.sequence.append(ResidualUnit(f_in,f_out,kernel_size))

    def call(self,x,training = False, mask = None):
        for unit in self.sequence:
            x = unit(x,training = training)
        return x

#모델 정의

In [5]:
class ResNet(tf.keras.Model):
    def __init__(self):
        super(ResNet,self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(8,(3,3),padding = 'same',activation='relu')#이렇게 해서 feature map을 한번 뽑아놔야 더 효과적이다
        #28x28x8
        self.res1 = ResnetLayer(8,(16,16),(3,3))#28x28x16
        self.pool1 = tf.keras.layers.MaxPool2D((2,2))

        self.res2 = ResnetLayer(16,(32,32),(3,3))#14x14x32
        self.pool2 = tf.keras.layers.MaxPool2D((2,2))#7x7x32

        self.res3 = ResnetLayer(32,(64,64),(3,3))#7x7x64
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128,activation='relu')
        self.dense2 = tf.keras.layers.Dense(10,activation='softmax')


    def call(self,x,training=False,mask=None):
        x = self.conv1(x)
        #vgg랑 다른점 training att를 직접 사용한다.
        #batchnormalization은 inference와 training에서 다르게 동작하기 떄문
        x = self.res1(x,training = training)
        x = self.pool1(x)

        x = self.res2(x,training = training)
        x = self.pool2(x)

        x = self.res3(x,training = training)

        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

#루프 정의

In [6]:
@tf.function
def train_step(model,images,labels,loss_object,optimizer,train_loss,train_accuracy):
  with tf.GradientTape() as tape:
    predictions = model(images,training = True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels,predictions)
  
@tf.function
def test_step(model,images,labels,loss_object,test_loss,test_accuracy):
  predictions = model(images, training = False)

  t_loss = loss_object(labels,predictions)
  test_loss(t_loss)
  test_accuracy(labels,predictions)

In [7]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test,y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0 #이렇게하면 float64로 됨

x_train = x_train[...,tf.newaxis].astype(np.float32)
x_test = x_test[...,tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)


11493376/11490434 [==============================] - 1s 0us/step


In [8]:
model = ResNet()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

In [9]:
for epoch in range(EPOCHS):
  for images,labels in train_ds:
    train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

  for test_images, test_labels in test_ds:
    test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

  template = 'Epoch:{}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch:1, Loss: 0.14802570641040802, Accuracy: 95.94667053222656, Test Loss: 0.08743691444396973, Test Accuracy: 97.37999725341797
Epoch:2, Loss: 0.06655445694923401, Accuracy: 98.12833404541016, Test Loss: 0.06702382117509842, Test Accuracy: 98.05999755859375
Epoch:3, Loss: 0.04997371882200241, Accuracy: 98.61000061035156, Test Loss: 0.09407401084899902, Test Accuracy: 97.38999938964844
Epoch:4, Loss: 0.042143601924180984, Accuracy: 98.78666687011719, Test Loss: 0.05169117823243141, Test Accuracy: 98.54999542236328
Epoch:5, Loss: 0.033962976187467575, Accuracy: 99.02166748046875, Test Loss: 0.03856852650642395, Test Accuracy: 98.87999725341797
Epoch:6, Loss: 0.02993639186024666, Accuracy: 99.1433334350586, Test Loss: 0.058003515005111694, Test Accuracy: 98.36000061035156
Epoch:7, Loss: 0.02560691349208355, Accuracy: 99.26499938964844, Test Loss: 0.040263619273900986, Test Accuracy: 98.94000244140625
Epoch:8, Loss: 0.02424672432243824, Accuracy: 99.28333282470703, Test Loss: 0.033103771